# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании. Задача - разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

**Проверим данные на наличие пропусков и аномалий.**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
display(data.head())
display(data.info())
display(data.describe())
print('кол-во строк дубликатов =', data.duplicated().sum())
data.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


кол-во строк дубликатов = 153


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


**Обнаружено в данных 3% дубликатов от общего количества данных. И положительная сильная корреляция между признаком `Возраст` и целевым признаком `Страховые выплаты`. Удалим дубликаты ниже.**

In [3]:
data = data.drop_duplicates()
print('кол-во строк дубликатов =', data.duplicated().sum())

кол-во строк дубликатов = 0


**Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.**

**Целевой признак: количество страховых выплат клиенту за последние 5 лет.**

**Также видим, что тип признаков `Возраст` и `Зарплата` float, хотя для данных признаков более подходящим будет целочисленный тип, приведем данные признаки к целочисленному типу.**

**Для удобства дальнейшей работы переведем названия признаков на английский язык и используем "змеиный регистр".**

In [4]:
data.columns = ['gender', 'age', 'salary', 'family_members', 'insurance_claim']
data = data.astype(int)
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   gender           4847 non-null   int64
 1   age              4847 non-null   int64
 2   salary           4847 non-null   int64
 3   family_members   4847 non-null   int64
 4   insurance_claim  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


None

**Выводы раздела 1.**

**В данных было обнаружено 3% дубликатов, дубликаты были удалены.**

**В данных положительная сильная корреляция между признаком `Возраст` и целевым признаком `Страховые выплаты`.**

**Пропуски в данных отсутствуют.**

**Типы данных всех признаков приведены к целочисленному.**

**Названия признаков, включая целевой, приведены к змеиному регистру и англ. наименованиям.**

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Признаки умножают на обратимую матрицу. Определим, изменится ли качество линейной регрессии (её можно обучить заново).**

$$
w = (X^T X)^{-1} X^T y
$$

**Качество линейной регрессии не изменится.**

**Обоснование:**

Т.к. указано, что мы умножаем признаки на обратимую матрицу, то исходя из критерия обратимости матриц можно сделать вывод, что умножение происходит на квадратную матрицу.

Второй факт:операция умножения двух матриц выполнима только в том случае, если число столбцов в первом сомножителе равно числу строк во втором; в этом случае говорят, что матрицы согласованы. Значит в нашем случае мы можем допускать умножение признаков только на квадратную матрицу, у которой высота и ширина равны ширине (количеству признаков) в матрице признаков.

**Исходная формула обучения (вектор весов) и предсказаний:**

$$
w = (X^T X)^{-1} X^T y
$$

$$
a = X w
$$

**Рассмотрим, что будет происходить в формуле обучения при умножении матрицы признаков на квадратную матрицу P:**
$$
w_p = (X_p^T X_p)^{-1} X_p^T y
$$

**Где:**  $ X_p = X P $

$$
w_p = ((X P)^T  (X P))^{-1} (X P)^T y
$$

**Раскроем скобки выражения выше по правилам преобразования с матрицами, получим:**
$$
w_p = (P^T X^T X P)^{-1} (P^T X^T) y
$$

**Сгруппируем в выражении выше скобками группы произведений матриц и отдельных матриц, получим:**
$$
w_p = ((P^T) (X^T X) (P))^{-1} P^T X^T y
$$

**Раскроем скобки, которые стоят под "степенью" (-1) в выражении выше, по правилам преобразования с матрицами, получим:**
$$
w_p = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

**Произведение транспонированной матрицы P на обратную транспонированной согласно свойстам матриц дает единичную матрицу E, умножая на которую другую матрицу, мы получаем аналогичную ей. Поэтому можно это просто сократить, получим следующее:**
$$
w_p = P^{-1} (X^T X)^{-1} X^T y
$$

**Теперь запишем формулу для нахождения $а_p$ - предсказаний целевого признака:**
$$
a_p = X_p w_p
$$

**Подставим в данную формулу найденное выше выражение для $w_p$, получим:**
$$
a_p = (X P) (P^{-1} (X^T X)^{-1} X^T y)
$$

**Раскроем скобки в выражении выше, получим:**
$$
a_p = X P P^{-1} (X^T X)^{-1} X^T y
$$

**Произведение матрицы P на обратную согласно свойстам матриц дает единичную матрицу E, умножая на которую другую матрицу, мы получаем аналогичную ей. Поэтому можно это просто сократить, получим следующее:**
$$
a_p = X (X^T X)^{-1} X^T y = X w = a
$$

**Итого получили:**
$$
a_p = a
$$

**Что доказывает, что предсказания будут аналогичными в случае умножения признаков на обратимую матрицу. ЧТД.**

## Алгоритм преобразования

**Алгоритм**

Исходя из доказательства выше, домножая признаки на обратимую матрицу, мы не ухудшаем качество линейной регрессии.

Критерий обратимости: матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель не равен нулю. Для неквадратных матриц и вырожденных матриц обратных матриц не существует.

Соответственно, необходимо сгенерировать такую квадратную матрицу, размерность которой совпадает с количеством признаков в исходных данных и определитель которой не равен 0. При этом сгенерированная матрица не должна быть единичной, иначе при умножении на нее исходной матрицы признаков мы получим ту же матрицу и преобразование данных не произойдет, задача защиты данных будет не провалена.

Структурированно алгоритм можно записать следующим образом:
1. Сгенерировать случайную квадратную матрицу размером M x M, где M это количество признаков в исходном датасете.
2. Проверить сгенерированную матрицу на обратимость, сравнив ее определитель с нулём.
3. Обучим модель линейной регрессии с помощью исходной матрицы признаков.
4. Используя обученную модель предскажем значения целевого признака на основе исходных признаков.
5. Вычислим метрику качества R2 для данной обученной модели линейной регрессии.
6. Умножим исходную матрицу признаков на сгенерированную матрицу, полученную матрицу назовем кодированная матрица признаков.
7. Переобучим модель линейной регрессии с помощью кодированной матрицы признаков.
8. Используя переобученную модель предскажем значения целевого признака на основе кодированных признаков.
9. Вычислим метрику качества R2 для переобученной модели линейной регрессии. 
10. Сравним метрику R2 исходной модели линейной регрессии и модели, обученной на кодированных признаках.

**Обоснование**

В разделе 2 было математически доказано, что умножение матрицы признаков на обратимую матрицу не меняет предсказания и, соответственно, не ухудшает качество линейной регрессии.

## Проверка алгоритма

Запрограммируем алгоритм, применив матричные операции. Далее проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Для этого применим метрику R2.

In [5]:
features = data.drop('insurance_claim', axis=1)
print('features.head before')
display(features.head())
target = data['insurance_claim']
model = LinearRegression()
model.fit(features, target)
predictions_before = model.predict(features)
score_before = r2_score(target, predictions_before)
print(f'R2 score before encryption = {score_before}')

singular = lambda m: np.linalg.det(m) == 0 #check if the matrix determinant = 0
P = np.random.random([features.shape[1],features.shape[1]]) #create square matrix with random elements from [0;1)
if not singular(P):
    features_encrypted = features.to_numpy() @ P # multiply initial features with the square random matrix
    print('\n\n\nencryption matrix')
    print(P)
    print('\n\n\nfeatures.head after encryption')
    display(pd.DataFrame(features_encrypted, columns = features.columns).head())
    model.fit(features_encrypted, target)
    predictions_after = model.predict(features_encrypted)
    score_after = r2_score(target, predictions_after)
    print(f'R2 score after encryption = {score_after}')
    print('\n\n\nr2_score difference:', score_after - score_before)

features.head before


,gender,age,salary,family_members
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


R2 score before encryption = 0.4302010046633359



encryption matrix
[[0.26130595 0.65787401 0.29584038 0.76437669]
 [0.84355721 0.28586644 0.30151503 0.79294882]
 [0.19892611 0.5234803  0.79208387 0.71328965]
 [0.40760029 0.73508471 0.72627317 0.40646642]]



features.head after encryption


,gender,age,salary,family_members
0,9901.989821,25977.736450,39300.744049,35412.848431
1,7598.403422,19906.136408,30113.782923,27141.888848
2,4201.911475,11001.376464,16642.505150,15002.078186
3,8313.748700,21836.601948,33037.681629,29761.643302
4,5215.852386,13671.498010,20682.127198,18639.826833


R2 score after encryption = 0.43020100466333566



r2_score difference: -2.220446049250313e-16


**Полученный результат в разности двух метрик качества близкий к нулю доказывает, что данный алгоритм шифрования работает корректно и не ухудшает качество линейной регрессии.**

## Вывод

**Исходный датасет состоял из 4 признаков и 1 целевого признака. Было удалено 3% от всех данных дубликатов, изменен тип всех признаков на целочисленный, приведены названия всех признаков к змеиному регистру и к английской версии для простоты дальнейшей работы.**

**Было доказано математически, что умножение матрицы исходных признаков на обратимую матрицу не ухудшает качество модели линейной регрессии, при этом таким образом можно закодировать первоначальные значения признаков, т.к. они являются конфиденциальной информацией.**

**Далее на практике с помощью кода был продемонстрирован пример того, как при умножении исходной матрицы признаков на случайную обратимую матрицу мы получили одинаковое значение метрики R2 линейной регрессии, переобученной на данных закодированных признаках. Было показано, что указанный метод шифрования не ухудшает качество линейной регрессии - значение метрик R2 было аналогичным как для модели, обученной на исходных признаках, так и для модели, обученной на кодированных признаках - и, соответственно данный метод шифрования данных может быть использован в работе.**